# Periodogram viewer for HOYS lightcurves

* ***Shift + Enter on a code cell to run it and advance to the next cell.***

* ***If changing sliders, rerun the code block below the sliders***

* ***Plot windows are interactive, can use mouse to select and zoom into ranges***

In [2]:
#load required modules
import os
import pandas as pd
import astropy
import numpy
import math
from astropy.time import Time
from astropy.timeseries import LombScargle
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import iplot
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display,clear_output

In [5]:
#####temporary test functions for binder environment######
os.system('mkdir montage_corr')

0

In [6]:
os.system('rm -rf montage_corr')

0

### List available lightcurve data and select from dropdown list
* If data has been uploaded to the binder notebook main page, this may need refreshing

In [3]:
#read HOYS csv file
#lc_folder = "/content/drive/My Drive/HOYS_Colab/HOYS_period/light_curve_csv_files/"
lc_folder = "./light_curve_csv_files/"
star_list=os.listdir(lc_folder)

lc_select = widgets.Dropdown(
    options=sorted(star_list),
    description='Select a light curve file:',
    style = {'description_width': 'initial'}
)
display(lc_select)



Dropdown(description='Select a light curve file:', options=('.DS_Store', 'lightcurve_2MASSJ20494917+4410462.tx…

### Display head and tail of csv file containing all of the data

In [4]:
print('selected datafile:',lc_select.value)
lc_data = pd.read_csv(os.path.join(lc_folder,lc_select.value),comment='#',delimiter=' ')
#view file head and tail
lc_data

selected datafile: lightcurve_2MASSJ20494917+4410462.txt


,id,calibrated_magnitude,calibrated_error,magnitude_rms_error,x,y,alpha_j2000,delta_j2000,fwhm_world,flags,magnitude,observation_id,filter,original_filter,date,user_id,device_id,target,fits_id
0,141209533,16.9169,0.138112,0.0446,2783.874268,2831.259521,312.453443,44.178961,0.001205,0.0,17.1059,14601,V,Visual,2.458745e+06,7,2,[118] IC5070_incl_201_V2492Cyg,14206
1,66408850,15.1992,0.048892,0.0112,2794.438232,2785.382080,312.453446,44.179551,0.001228,3.0,15.9083,5600,I,I-Band,2.458346e+06,7,2,[118] IC5070_incl_201_V2492Cyg,5453
2,60322416,17.2320,0.168609,0.0472,1275.067871,1263.462891,312.453452,44.179037,0.000740,0.0,17.8271,4628,V,Visual,2.458314e+06,7,2,[118] IC5070_incl_201_V2492Cyg,4470
3,192175370,16.8221,0.110303,0.0230,2821.169922,2935.495117,312.453457,44.179166,0.000967,0.0,16.9552,32219,V,Visual,2.459060e+06,7,2,[118] IC5070_incl_201_V2492Cyg,31482
4,192184082,16.8221,0.110303,0.0230,2821.169922,2935.495117,312.453457,44.179166,0.000967,0.0,16.9552,32219,V,Visual,2.459060e+06,7,2,[118] IC5070_incl_201_V2492Cyg,31482
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3387,70138979,16.7623,0.000000,0.0615,1186.303467,976.338501,312.455742,44.179652,0.001228,512.0,15.7262,6997,V,V,2.458369e+06,31,39,[118] IC5070_incl_201_V2492Cyg,7011
3388,158784840,16.7935,0.159803,0.0489,1618.546509,439.825897,312.455743,44.179712,0.000599,0.0,16.5482,6762,V,V,2.458362e+06,41,51,[118] IC5070_incl_201_V2492Cyg,6431
3389,76581444,15.7276,0.000000,0.1480,1161.817139,933.094788,312.455752,44.179229,0.001694,512.0,15.2722,7688,V,V,2.458386e+06,31,39,[118] IC5070_incl_201_V2492Cyg,7560
3390,66645463,18.5902,0.000000,0.1544,1600.849487,434.995392,312.455753,44.179904,0.000674,512.0,17.8630,5624,B,B,2.458342e+06,41,51,[118] IC5070_incl_201_V2492Cyg,5480


In [48]:
filter_list=numpy.unique(lc_data['filter'])

filter_select = widgets.Dropdown(
    options=sorted(filter_list),
    value='V',
    description='Select a filter',
)
display(filter_select)

date_range_sel=widgets.FloatRangeSlider(
    value=[min(lc_data['date']),max(lc_data['date'])],
    min=min(lc_data['date']),
    max=max(lc_data['date']),
    step=0.1,
    description='Date range [jd]:',
    readout_format='.1f',
    layout={'width': '600px'},
    style = {'description_width': 'initial'}

)

display(date_range_sel)

cal_error_sel=widgets.FloatSlider(
    value=0.25,
    min=0.1,
    max=0.4,
    step=0.01,
    description='Calibrated error:',
    layout={'width': '600px'},
    style = {'description_width': 'initial'}
)

display(cal_error_sel)

med_window_sel=widgets.IntSlider(
    value=50,
    min=10,
    max=365,
    step=5,
    description='Median filter window [days]:',
    layout={'width': '600px'},
    style = {'description_width': 'initial'}
)

display(med_window_sel)




Dropdown(description='Select a filter', index=5, options=('B', 'HA', 'I', 'R', 'U', 'V'), value='V')

FloatRangeSlider(value=(2457275.45849, 2459074.57217517), description='Date range [jd]:', layout=Layout(width=…

FloatSlider(value=0.25, description='Calibrated error:', layout=Layout(width='600px'), max=0.4, min=0.1, step=…

IntSlider(value=50, description='Median filter window [days]:', layout=Layout(width='600px'), max=365, min=10,…

### Select desired filter and use sliders to refine data

In [49]:
#select data from specifed filter and remove bad data
band=filter_select.value
cal_error=cal_error_sel.value
med_window=med_window_sel.value
date_range=date_range_sel.value

i_data=lc_data[(lc_data['filter']==band) & (lc_data['flags'] <= 4) & (lc_data['calibrated_error'] < cal_error) & (lc_data['date'] > date_range[0]) & (lc_data['date'] < date_range[1]) & (lc_data['calibrated_magnitude'] > 0.0) & (lc_data['calibrated_error'] > 0.0) & (lc_data['fwhm_world'] > 0.0) & (lc_data['fwhm_world'] < 9.0/3600.0)]


#median filter the lightcurve over a time window
window = med_window #half the time window in days
mags = numpy.array(i_data['calibrated_magnitude'])
times = numpy.array(i_data['date'])
med = numpy.zeros(len(i_data))
for i in range(0,len(med)):
    check = numpy.where( numpy.abs(times[i] - times) < window )
    if (len(check[0]) > 0):
        med[i] = numpy.median(mags[check[0]])
filtered_mag=pd.Series(i_data['calibrated_magnitude'] - med + numpy.median(i_data['calibrated_magnitude']),name='filtered_mag')
i_data=pd.concat([i_data,filtered_mag],axis=1)

mag_range_sel=widgets.FloatRangeSlider(
    min=min(i_data['filtered_mag']),
    max=max(i_data['filtered_mag']),
    step=0.01,
    value=[min(i_data['filtered_mag']),max(i_data['filtered_mag'])],
    layout={'width': '600px'},
    description='Magnitude range [mag]',
    style = {'description_width': 'initial'}

)

display(mag_range_sel)

FloatRangeSlider(value=(13.84825, 15.18045), description='Magnitude range [mag]', layout=Layout(width='600px')…

### Select magnitude range, and plot lightcurve below, if changing magnitude range, rerun lightcurve plot below

In [53]:
mag_range=mag_range_sel.value
mag_data=i_data[(i_data['filtered_mag'] > mag_range[0]) & (i_data['filtered_mag'] < mag_range[1])]

mag_data=mag_data.sample(25)

#plot lightcurve
fig1=px.scatter(mag_data,x=mag_data.date,y=mag_data.filtered_mag)
fig1['layout']['yaxis']['autorange'] = "reversed"
iplot(fig1)

### Plot the periodogram and determine the best period

In [54]:
#generate and plot periodogram
#jd=Time(i_data.date,format='jd') #was using the astropy time units for the date
jd=mag_data.date
ls=LombScargle(jd,mag_data.filtered_mag)
frequency,power=ls.autopower()
fig2=px.line(x=1/frequency,y=power,labels={'x':'Period [days]','y':'Power'})
fig2.update_xaxes(range=[0, (max(jd)-min(jd))/4.]) #added max range for plot, change if a better range is more useful
iplot(fig2)
#best_period = 1/frequency[numpy.argmax(power[numpy.where(1./frequency < 1000.0)])]
best_period = 1/frequency[numpy.argmax(power)]
print('best period:',numpy.round(best_period,5),'days')

best period: 35.52749 days


### Plot the phase folded light curve and use the sliders to fit the best periodic function


In [61]:
time = mag_data['date']
phase = numpy.zeros(len(time))

#determine start values for the paramers from the data
offset0 = numpy.median(mag_data['filtered_mag'])
amp0 = numpy.std(mag_data['filtered_mag']) * 1.414
period0 = best_period
#trying to add interactive sliders for the sine-function - needs period, amp, offset and phase0 to be variable
def view_image(period=period0, amp=amp0, offset=offset0, phase0=0.5):

    phase = numpy.mod( time/period , 1.0 )
    #add a fitted sine-function
    amp = amp
    phase0 = phase0
    offset= offset
    fit = offset + amp * numpy.sin(2.0*math.pi*(-phase0 + phase ))

    #create dataframe with the phase, magnitude and fit values
    #use append to expand the size of it all to include phase+1.0
    df_phase=pd.DataFrame({'phase':phase.append(phase+1.0), 'data':mag_data['filtered_mag'].append(mag_data['filtered_mag']), 'fit':fit.append(fit)})
    #melt the dataframe by phase as the id variable so that the data can be plot and coloured by whether it's the data or the fit
    df_melt = df_phase.melt(id_vars='phase', value_vars=['data', 'fit'],value_name='filtered_mag')

    fig4=px.scatter(df_melt,x='phase', y='filtered_mag',color='variable',template="plotly_white")
    fig4['layout']['yaxis']['autorange'] = "reversed"
    fig4.update_traces(mode='markers', marker_line_width=3, marker_size=15)
    
    fig4.update_layout(
    autosize=False,
    width=900,
    height=600,)

    iplot(fig4)

interact(view_image, period=widgets.FloatSlider(min=period0-0.05, max=period0+0.05,step= 0.0001,value=period0,layout={'width': '800px'},readout_format='.5f'), 
         amp=widgets.FloatSlider(min=0, max=amp0+0.3, step=0.001,value=amp0,layout={'width': '800px'}), 
         offset=widgets.FloatSlider(min=offset0-0.2,max=offset0+0.2,step=0.001,layout={'width': '800px'},value=offset0,readout_format='.3f'), 
         phase0=widgets.FloatSlider(min=0,max=1,step=0.01,value=0.5,layout={'width': '800px'}))


interactive(children=(FloatSlider(value=35.527491372519435, description='period', layout=Layout(width='800px')…

<function __main__.view_image(period=35.527491372519435, amp=0.09317904437659376, offset=14.137749999999999, phase0=0.5)>